In [1]:
import pandas as pd
import numpy as np

/var/folders/47/sx3slfp54xv_8syv33c8kzz80000gn/T/ipykernel_12631/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# create sample classification dataset with 1000 rows and 20 features
from sklearn.datasets import make_classification
from random import randrange


X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# X, y = make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0, random_state=1)
print(X.shape, y.shape)

# converting to dataframes
X = pd.DataFrame(X)
y = pd.Series(y)


def subsample(X, y, ratio=1):
    temp_X = list()
    temp_y = list()

    n_sample = round(len(X) * ratio)

    while len(temp_X) < n_sample:
        index = randrange(len(X))

        temp_X.append(X.iloc[index])
        temp_y.append(y.iloc[index])

    return temp_X, temp_y

num_bags = 3
ratio = 0.6

bags_X = list()
bags_y= list()

for i in range(num_bags):  # Corrected the range to iterate over all bags
    sample_X, sample_y = subsample(X, y, ratio)

    bags_X.append(sample_X)
    bags_y.append(sample_y)
    


(1000, 20) (1000,)


In [28]:
for i, bag in enumerate(bags_X, start=1):
    globals()[f'X_{i}'] = pd.DataFrame(bag)

for i, bag in enumerate(bags_y, start=1):
    globals()[f'y_{i}'] = pd.DataFrame(bag)


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the base classifier (you can use any classifier here)
base_classifier = DecisionTreeClassifier(random_state=42)

# Define the bagging classifier
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=10, random_state=42)

# Train the bagging classifier
bagging_classifier.fit(X_train, y_train)

# Make predictions
y_pred = bagging_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [4]:
# preprocessing to make all inputs between 0 and 1
from sklearn.preprocessing import MinMaxScaler

scaler= MinMaxScaler()
# X_1= scaler.fit_transform(X_1)
# X_2= scaler.fit_transform(X_2)
# X_3= scaler.fit_transform(X_3)

X= scaler.fit_transform(X)



In [6]:
# logistic regression model
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(solver="liblinear", random_state=1)
# lr_i= lr.fit(X, y)

# print("Init Score: ", lr_i.score(X, y))

# print("Scores after bagging")
# # fit lr to each bag
# lr_1= lr.fit(X_1, y_1)
# lr_2= lr.fit(X_2, y_2)
# lr_3= lr.fit(X_3, y_3)


# print(lr_1.score(X_1, y_1))
# print(lr_2.score(X_2, y_2))
# print(lr_3.score(X_3, y_3))


# vc_lr = VotingClassifier([("clf1", lr_1), ("clf2", lr_2), ("clf3", lr_3)], voting="soft")
# vc_lr.fit(X, y)
# print("Final Score on entire dataset:", vc_lr.score(X, y))
                       

In [7]:
# random forest model
from sklearn.ensemble import RandomForestClassifier

rf= RandomForestClassifier(n_estimators=100, random_state=1)

# rf.fit(X, y)

# # fit rf to each bag
# rf_1= rf.fit(X_1, y_1)
# rf_2= rf.fit(X_2, y_2)
# rf_3= rf.fit(X_3, y_3)

# print(rf_1.score(X_1, y_1))
# print(rf_2.score(X_2, y_2))
# print(rf_3.score(X_3, y_3))


In [8]:
# multinomial nb model
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

# fit nb to each bag
# nb_1= nb.fit(X_1, y_1)
# nb_2= nb.fit(X_2, y_2)
# nb_3= nb.fit(X_3, y_3)

# print(nb_1.score(X_1, y_1))
# print(nb_2.score(X_2, y_2))
# print(nb_3.score(X_3, y_3))



In [44]:
# ensemble
from sklearn.ensemble import VotingClassifier

vc_afterbagging = VotingClassifier([("clf1", lr_1), ("clf2", lr_2), ("clf3", lr_3), ("clf4", rf_1), ("clf5", rf_2), ("clf6", rf_3), ("clf7", nb_1), ("clf8", nb_2), ("clf9", nb_3)])

cross_val_score(vc_afterbagging, X, y).mean()

0.8629999999999999

In [45]:
vc_withoutbagging = VotingClassifier([("clf1", lr), ("clf2", rf), ("clf3", nb)])
cross_val_score(vc_withoutbagging, X, y).mean()

0.8629999999999999

In [25]:
# finetuning votingclassifier
params= {'voting': ['hard', 'soft'], 
         'weights': [[1,1,1], [1,2,1], [1,1,2], [1,2,2]],
         'clf1__C': [0.01, 0.1, 1, 10, 100],
         'clf2__n_estimators': [10, 100, 1000],
         'clf3__alpha': [0.01, 0.1, 1, 10, 100]}


In [26]:
# grid
from sklearn.model_selection import GridSearchCV

grid= GridSearchCV(vc, params, cv=5, n_jobs=-1)
grid.fit(X, y)
grid.best_params_

{'clf1__C': 0.01,
 'clf2__n_estimators': 1000,
 'clf3__alpha': 100,
 'voting': 'soft',
 'weights': [1, 2, 1]}

In [9]:
# voring classifier with best parameters
vc = VotingClassifier(estimators=[("clf1", LogisticRegression(C=0.01, solver="liblinear", random_state=1)), 
                       ("clf2", RandomForestClassifier(n_estimators=1000, random_state=1)), 
                       ("clf3", MultinomialNB(alpha=0.01))], voting="soft", weights=[1,2,1])

cross_val_score(vc, X, y).mean()

0.9149999999999998

In [40]:
x_sample= X[100]
vc.fit(X,y)
vc.predict(x_sample.reshape(1, -1))

array([1])

In [32]:
# to do create custom estimator

from sklearn.utils.estimator_checks import check_estimator
from sklearn.svm import LinearSVC

check_estimator(LinearSVC())

/Users/jyotit-kaushal/miniconda3/envs/capstone_ensemble/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/jyotit-kaushal/miniconda3/envs/capstone_ensemble/lib/python3.12/site-packages/sklearn/utils/estimator_checks.py:652: SkipTestWarning: Skipping check_sample_weights_invariance for LinearSVC: zero sample_weight is not equivalent to removing samples
  warnings.warn(str(exception), SkipTestWarning)
/Users/jyotit-kaushal/miniconda3/envs/capstone_ensemble/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/jyotit-kaushal/miniconda3/envs/capstone_ensemble/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warning

In [34]:
from sklearn.base import BaseEstimator

class customEstimator(BaseEstimator):
    def __init__(self, C=1):
        self.C = C

    def fit(self, X, y):
        self.clf = LinearSVC(C=self.C)
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)
    
    def score(self, X, y):
        return cross_val_score(self.clf, X, y).mean()

In [37]:
ce= customEstimator()
ce.fit(X, y)
ce.score(X, y)


/Users/jyotit-kaushal/miniconda3/envs/capstone_ensemble/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/jyotit-kaushal/miniconda3/envs/capstone_ensemble/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/jyotit-kaushal/miniconda3/envs/capstone_ensemble/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/jyotit-kaushal/miniconda3/envs/capstone_ensemble/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change f

0.8699999999999999

In [ ]:
# param setting for ensembling

# clf1 raw-gat-st
# clf2 raw-pc

params= {'voting': ['hard', 'soft'], 
        'weights': [[1,1,1], [1,2,1], [1,1,2], [1,2,2]],
        'clf1__lr': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.90],
        'clf1__momentum': [1e-4, 1e-3, 1e-2, 0.90],
        'clf1__lr': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.90],
        'clf1__momentum': [1e-4, 1e-3, 1e-2, 0.90],
        'clf2__n_estimators': [10, 100, 1000],
        'clf3__alpha': [0.01, 0.1, 1, 10, 100]}

In [10]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the base classifier (you can use any classifier here)
base_classifier = DecisionTreeClassifier(random_state=42)

# Define the bagging classifier
# bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=10, random_state=42)
bagging_classifier = BaggingClassifier(estimator=vc, n_estimators=3, max_samples=0.6, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0)

# Train the bagging classifier
bagging_classifier.fit(X_train, y_train)

# Make predictions
y_pred = bagging_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.925


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils import resample

# Define your custom neural network
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define training parameters
num_models = 5
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# Define your dataset (dummy data for illustration)
X_train = torch.randn(1000, 10)  # Assuming 1000 samples with 10 features
y_train = torch.randint(0, 2, (1000,))  # Binary classification labels

# Convert dataset to PyTorch DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize a list to store the trained models
models = []

# Train multiple models with bagging
for i in range(num_models):
    # Create a new instance of the custom neural network
    model = CustomNN()
    
    # Define loss function and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Train the model
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
    
    # Append the trained model to the list
    models.append(model)

# Define a function to make predictions using the ensemble of models
def predict_ensemble(models, inputs):
    predictions = torch.zeros(len(inputs))
    for model in models:
        outputs = model(inputs)
        predictions += outputs.squeeze().detach().sigmoid().round()
    return (predictions / len(models)).round()

# Example usage:
X_test = torch.randn(100, 10)  # Assuming 100 samples for testing
y_test= torch.randn(100, 1)
predictions = predict_ensemble(models, X_test)
print(predictions)

# testing the accuracy of predictions
def calculate_accuracy(predictions, targets):
    correct = (predictions == targets).sum().item()
    total = len(targets)
    accuracy = correct / total
    return accuracy

# Calculate predictions


# Calculate accuracy
accuracy = calculate_accuracy(predictions, y_test)
print("Accuracy:", accuracy)


tensor([0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
        1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
        0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1.,
        0., 1., 1., 0., 0., 0., 0., 1., 1., 1.])
Accuracy: 0.0


In [ ]:
## make an ensemble classifier based on decision trees ##
class BaggedTreeClassifier(object):
    #initializer
    def __init__(self,n_elements=100):
        self.n_elements = n_elements
        self.models     = []
    
    #destructor
    def __del__(self):
        del self.n_elements
        del self.models
        
    #private function to make bootstrap samples
    def __make_bootstraps(self,data):
        #initialize output dictionary & unique value count
        dc   = {}
        unip = 0
        #get sample size
        b_size = data.shape[0]
        #get list of row indexes
        idx = [i for i in range(b_size)]
        #loop through the required number of bootstraps
        for b in range(self.n_elements):
            #obtain boostrap samples with replacement
            sidx   = np.random.choice(idx,replace=True,size=b_size)
            b_samp = data[sidx,:]
            #compute number of unique values contained in the bootstrap sample
            unip  += len(set(sidx))
            #obtain out-of-bag samples for the current b
            oidx   = list(set(idx) - set(sidx))
            o_samp = np.array([])
            if oidx:
                o_samp = data[oidx,:]
            #store results
            dc['boot_'+str(b)] = {'boot':b_samp,'test':o_samp}
        #return the bootstrap results
        return(dc)
